In [ ]:
#Metadata

In [2]:
from qiime2 import Metadata
from urllib import request

url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/020-tutorial-upstream/020-metadata/sample-metadata.tsv'
fn = 'sample-metadata.tsv'
request.urlretrieve(url, fn)
sample_metadata_md = Metadata.load(fn)

In [ ]:
import qiime2.plugins.metadata.actions as metadata_actions

metadata_summ_1_viz, = metadata_actions.tabulate(input=sample_metadata_md,)

In [6]:
#Demultiplex

In [ ]:
import zipfile

url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/020-tutorial-upstream/030-importing/data_to_import.zip'
fn = 'data_to_import.zip'
request.urlretrieve(url, fn)
with zipfile.ZipFile(fn) as zf:
    zf.extractall('data_to_import')

In [ ]:
from q2_types.per_sample_sequences import CasavaOneEightSingleLanePerSampleDirFmt
from qiime2 import Artifact

demultiplexed_sequences = Artifact.import_data(
    'SampleData[PairedEndSequencesWithQuality]',
    'data_to_import',
    CasavaOneEightSingleLanePerSampleDirFmt,)

In [ ]:
import qiime2.plugins.demux.actions as demux_actions

demultiplexed_sequences_summ_viz, = demux_actions.summarize(
    data=demultiplexed_sequences,)

In [ ]:
#Denoise

In [ ]:
import qiime2.plugins.dada2.actions as dada2_actions

feature_table_0, asv_sequences_0, dada2_stats = dada2_actions.denoise_paired(
    demultiplexed_seqs=demultiplexed_sequences,
    trunc_len_f=204,
    trim_left_r=1,
    trunc_len_r=205,)

In [ ]:
stats_dada2_md_md = dada2_stats.view(Metadata)
dada2_stats_summ_viz, = metadata_actions.tabulate(
    input=stats_dada2_md_md,)

In [8]:
import qiime2.plugins.feature_table.actions as feature_table_actions

feature_table_0_summ_viz, = feature_table_actions.summarize(
    table=feature_table_0,
    sample_metadata=sample_metadata_md,)

asv_sequences_0_summ_viz, = feature_table_actions.tabulate_seqs(
    data=asv_sequences_0,)

In [ ]:
#Filter
#jupyter serverextension enable --py qiime2 --sys-prefix needed fro visualization

In [14]:
url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/030-tutorial-downstream/010-filtering/feature-table.qza'
fn = 'feature-table.qza'
request.urlretrieve(url, fn)
feature_table = Artifact.load(fn)

In [15]:
url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/030-tutorial-downstream/010-filtering/rep-seqs.qza'
fn = 'rep-seqs.qza'
request.urlretrieve(url, fn)
rep_seqs = Artifact.load(fn)

In [27]:
table_viz, = feature_table_actions.summarize(
    table=feature_table,
    sample_metadata=sample_metadata_md,)
table_viz.save('table.qzv')

rep_seqs_viz, = feature_table_actions.tabulate_seqs(
    data=rep_seqs,)
rep_seqs_viz.save('resp_seqs.qzv')

In [11]:
table_viz

<visualization: Visualization uuid: b0c159a8-df76-4fd4-9f8d-46d96c5abe8c>

In [15]:
rep_seqs_viz

<visualization: Visualization uuid: 509f7752-0f06-438b-b997-7f15968c06e7>

In [16]:
autofmt_table, = feature_table_actions.filter_samples(
    table=feature_table,
    metadata=sample_metadata_md,
    where='autoFmtGroup IS NOT NULL',)

In [17]:
autofmt_table.save('autofmt-table.qza')

'autofmt-table.qza'

In [18]:
autofmt_table_summ_viz, = feature_table_actions.summarize(
    table=autofmt_table,
    sample_metadata=sample_metadata_md,)

In [19]:
autofmt_table_summ_viz.save('autofmt-table-summ.qzv')

'autofmt-table-summ.qzv'

In [20]:
autofmt_table_summ_viz

<visualization: Visualization uuid: 13642732-6850-400d-8669-c6cc88641c90>

In [21]:
filtered_table_1, = feature_table_actions.filter_samples(
    table=autofmt_table,
    metadata=sample_metadata_md,
    where='DayRelativeToNearestHCT BETWEEN -10 AND 70',)

In [22]:
filtered_table_1.save('filtered_table_1.qza')

'filtered_table_1.qza'

In [23]:
filtered_table_2, = feature_table_actions.filter_features(
    table=filtered_table_1,
    min_samples=2,)

In [24]:
filtered_table_2.save('filtered_table_2.qza')

'filtered_table_2.qza'

In [25]:
filtered_sequences_1, = feature_table_actions.filter_seqs(
    data=rep_seqs,
    table=filtered_table_2,)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_types/feature_data/_transformer.py:258: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id_, seq in data.iteritems():


In [26]:
filtered_sequences_1.save('filtered_sequences_1.qza')

'filtered_sequences_1.qza'